# DosxTres con tapa abajo, en 3D

In [1]:
from folpy.utils.parser.parser import Parser
from folpy.semantics.classes import Quasivariety
from functools import reduce
from itertools import islice

Importamos el lattice N5 y la cadena de 2 elementos. Luego generamos la cuasivariedad generada por N5 y obtenemos el lattice que vamos a estudiar, en este caso una subálgebra del producto N5*C2.

In [38]:
N5 = Parser("models/N5.model").parse()
C2 = Parser("models/2chain.model").parse()

Q = Quasivariety([N5])
A_inicial = next((C2*C2*N5).substructures()).continous()[0]

print("Cantidad de SRI: %s (salvo isomorfismos)" % len(Q.generate_rsi())) 
A_inicial.draw()

Cantidad de SRI: 2 (salvo isomorfismos)


Generamos la descomposición subdirecta del lattice, y nos quedamos con el lattice isomorfo pero que es un producto subdirecto:

In [13]:
from utils import descomp_subdirecta
homeo = descomp_subdirecta(A_inicial, Q)

P = homeo.target
A = P.restrict(list(homeo.image()))
n = len(P.indexes())

print("Cantidad de factores: %s" % n)

for i in P.indexes():
    print("Cantidad de elementos del álgebra %s: %s" % (i,len(P.factors[i])) )

Cantidad de factores: 5
Cantidad de elementos del álgebra 0: 2
Cantidad de elementos del álgebra 1: 2
Cantidad de elementos del álgebra 2: 2
Cantidad de elementos del álgebra 3: 5
Cantidad de elementos del álgebra 4: 2


A partir de los factores de la descomposición, generamos las congruencias que dan esa descomposición, que van a ser las CMI del reticulado de congruencias de $A$.

In [14]:
deltas = {}
for i in P.indexes():
    deltas[i] = P.projection(i).composition(homeo).kernel()
    print("delta_%s: %s" % (i, deltas[i]))

delta_0: Congruence([|0, 1, 2, 3, 4, 5, 6, 7, 8, 9|, |10, 11, 12, 13, 14, 15|])
delta_1: Congruence([|0, 1, 10, 11|, |2, 3, 4, 5, 6, 7, 8, 9, 12, 13, 14, 15|])
delta_2: Congruence([|0, 2, 3, 6, 7, 10, 12, 13|, |1, 4, 5, 8, 9, 11, 14, 15|])
delta_3: Congruence([|0, 1|, |2, 3, 4, 5|, |6, 7, 8, 9|, |10, 11|, |12, 13, 14, 15|])
delta_4: Congruence([|0, 1, 2, 4, 6, 8, 10, 11, 12, 14|, |3, 5, 7, 9, 13, 15|])


In [15]:
for i in P.indexes():
    for j in P.indexes():
        print("(%s,%s): %s" % (i, j, deltas[i]<deltas[j]))

(0,0): False
(0,1): False
(0,2): False
(0,3): False
(0,4): False
(1,0): False
(1,1): False
(1,2): False
(1,3): False
(1,4): False
(2,0): False
(2,1): False
(2,2): False
(2,3): False
(2,4): False
(3,0): True
(3,1): True
(3,2): False
(3,3): False
(3,4): False
(4,0): False
(4,1): False
(4,2): False
(4,3): False
(4,4): False


Cuando analizamos la congruencia, vemos que las CMI de $A$ están ordenadas de la siguiente manera:

```
d0    d1
  \  /
   d3     d2    d4
```

Sabemos que las congruencias que tomemos para el sistema van a ser generadas por intersecciones de CMIs, por lo tanto, viendo las congruencias como conjuntos de índices que se corresponden con las CMIs, estos conjuntos son crecientes. Además queremos que la unión de nuestros conjuntos de índice cubra todos los índices, por lo tanto va a haber al menos un $F_i$ que tenga a los indices $0, 1 $ y $3$; otro $F_i$ que tenga a $2$ y otro $F_i$ que tenga a $4$.

Por lo tanto comenzamos tomando nuestro conjunto de congruencias $\{\delta_1, \delta_2\, \delta_3\}$; es decir, los conjuntos de indices $F_1 = \{0,1,3\}$; $F_2 = \{2\}$ y $F_3 = \{4\}$.

In [16]:
Fs = {}
Fs[1] = {0, 1, 3}
Fs[2] = {2}
Fs[3] = {4}

Vamos a definir algunos operadores a partir de $A$ y $P := \prod A_i$.

 - $A' := \{ \bar{s} \in P : \ \bar{s}|_{F_i} \in A|_{F_i} \ \forall i \in I \}$
 - $\bar{A_{ij}} := \{ \bar{a} \in P : \ a_{ij} \in A_{ij} \}$

Cómo estamos en una variedad con M, se puede ver que:

 - $A = \bigcap_{i,j\in H} \bar{A_{ij}}$, donde $H := \{ ij : ij \in I^2 \}$
 - $A' = \bigcap_{i,j\in H'} \bar{A_{ij}}$, donde $H' := \{ ij \in I^2 : \exist k \in I $ tal que $ ij \in F_k \}$ 

Es claro que $H' \subseteq H$, y por lo tanto $A \subseteq A'$. Si se cumple la igualdad ($A = A'$), tenemos que todo sistema tiene solución, sino existe alguno sin solución. 


In [18]:
from utils import dos_proyeccion_extendida

H = [(i,j) for i in range(n) for j in range(n) if i < j]

A_barra = P.universe
for (i, j) in H:
    A_barra = [x for x in A_barra if x in dos_proyeccion_extendida(homeo, i, j)]

assert(sorted(A_barra) == sorted(A.universe))

In [19]:
H_prima = [(i,j) for (i,j) in H if any(i in Fs[k] and j in Fs[k] for k in Fs.keys())]

A_barra_prima = P.universe
for (i, j) in H_prima:
    A_barra_prima = [x for x in A_barra_prima if x in dos_proyeccion_extendida(homeo, i, j)]

set(A_barra_prima).difference(set(A_barra))

{(0, 0, 0, 0, 1), (0, 0, 1, 0, 1), (1, 0, 0, 3, 1), (1, 0, 1, 3, 1)}

$A'$ tiene 4 elementos más que $A$, por lo que van a existir 4 sistemas que no tengan solución para $\{\delta_1, \delta_2, \delta_3\}$.


### Caso 1

 - $F_1 := \{ 0, 1, 3 \}$
 - $F_2 := \{ 2, 4 \}$

In [26]:
Fs = {}
Fs[1] = {0, 1, 3}
Fs[2] = {2, 4}
tita = reduce(lambda x, y: x & y, [deltas[i] for i in Fs[1]])
gama = reduce(lambda x, y: x & y, [deltas[i] for i in Fs[2]])

H_prima = [(i,j) for (i,j) in H if any(i in Fs[k] and j in Fs[k] for k in Fs.keys())]

A_barra_prima = P.universe
for (i, j) in H_prima:
    A_barra_prima = [x for x in A_barra_prima if x in dos_proyeccion_extendida(homeo, i, j)]

set(A_barra_prima).difference(set(A_barra))

{(0, 0, 0, 0, 1), (0, 0, 1, 0, 1), (1, 0, 0, 3, 1), (1, 0, 1, 3, 1)}

Siguen estando los mismos elementos.

In [27]:
tita, gama

(Congruence([|0, 1|, |2, 3, 4, 5|, |6, 7, 8, 9|, |10, 11|, |12, 13, 14, 15|]),
 Congruence([|0, 2, 6, 10, 12|, |3, 7, 13|, |1, 4, 8, 11, 14|, |5, 9, 15|]))

In [28]:
from folpy.semantics.lattices import LatticeQuotient

LatticeQuotient(A_inicial, gama).draw()
LatticeQuotient(A_inicial, tita).draw()

### Caso 2

 - $F_1 := \{ 0, 1, 2, 3 \}$
 - $F_2 := \{ 4 \}$

In [30]:
Fs = {}
Fs[1] = {0, 1, 2, 3}
Fs[2] = {4}
tita = reduce(lambda x, y: x & y, [deltas[i] for i in Fs[1]])
gama = reduce(lambda x, y: x & y, [deltas[i] for i in Fs[2]])

H_prima = [(i,j) for (i,j) in H if any(i in Fs[k] and j in Fs[k] for k in Fs.keys())]

A_barra_prima = P.universe
for (i, j) in H_prima:
    A_barra_prima = [x for x in A_barra_prima if x in dos_proyeccion_extendida(homeo, i, j)]

set(A_barra_prima).difference(set(A_barra))

{(0, 0, 0, 0, 1), (0, 0, 1, 0, 1), (1, 0, 0, 3, 1), (1, 0, 1, 3, 1)}

In [31]:
tita, gama

(Congruence([|0|, |1|, |2, 3|, |4, 5|, |6, 7|, |8, 9|, |10|, |11|, |12, 13|, |14, 15|]),
 Congruence([|0, 1, 2, 4, 6, 8, 10, 11, 12, 14|, |3, 5, 7, 9, 13, 15|]))

In [32]:
from folpy.semantics.lattices import LatticeQuotient

LatticeQuotient(A_inicial, gama).draw()
LatticeQuotient(A_inicial, tita).draw()

### Caso 3

 - $F_1 := \{ 0, 1, 3 \}$
 - $F_2 := \{ 0, 2 \}$
 - $F_2 := \{ 4 \}$

In [42]:
Fs = {}
Fs[1] = {0, 1, 3}
Fs[2] = {0, 2}
Fs[3] = {4}
tita = reduce(lambda x, y: x & y, [deltas[i] for i in Fs[1]])
gama = reduce(lambda x, y: x & y, [deltas[i] for i in Fs[2]])
landa = reduce(lambda x, y: x & y, [deltas[i] for i in Fs[3]])

H_prima = [(i,j) for (i,j) in H if any(i in Fs[k] and j in Fs[k] for k in Fs.keys())]

A_barra_prima = P.universe
for (i, j) in H_prima:
    A_barra_prima = [x for x in A_barra_prima if x in dos_proyeccion_extendida(homeo, i, j)]

set(A_barra_prima).difference(set(A_barra))

{(0, 0, 0, 0, 1), (0, 0, 1, 0, 1), (1, 0, 0, 3, 1), (1, 0, 1, 3, 1)}

In [43]:
tita, gama, landa

(Congruence([|0, 1|, |2, 3, 4, 5|, |6, 7, 8, 9|, |10, 11|, |12, 13, 14, 15|]),
 Congruence([|0, 2, 3, 6, 7|, |1, 4, 5, 8, 9|, |10, 12, 13|, |11, 14, 15|]),
 Congruence([|0, 1, 2, 4, 6, 8, 10, 11, 12, 14|, |3, 5, 7, 9, 13, 15|]))

In [44]:
from folpy.semantics.lattices import LatticeQuotient

LatticeQuotient(A_inicial, gama).draw()
LatticeQuotient(A_inicial, tita).draw()
LatticeQuotient(A_inicial, landa).draw()

### Caso 4

 - $F_1 := \{ 0, 1, 3 \}$
 - $F_2 := \{ 1, 2 \}$
 - $F_2 := \{ 4 \}$

In [45]:
Fs = {}
Fs[1] = {0, 1, 3}
Fs[2] = {1, 2}
Fs[3] = {4}
tita = reduce(lambda x, y: x & y, [deltas[i] for i in Fs[1]])
gama = reduce(lambda x, y: x & y, [deltas[i] for i in Fs[2]])
landa = reduce(lambda x, y: x & y, [deltas[i] for i in Fs[3]])

H_prima = [(i,j) for (i,j) in H if any(i in Fs[k] and j in Fs[k] for k in Fs.keys())]

A_barra_prima = P.universe
for (i, j) in H_prima:
    A_barra_prima = [x for x in A_barra_prima if x in dos_proyeccion_extendida(homeo, i, j)]

set(A_barra_prima).difference(set(A_barra))

{(0, 0, 0, 0, 1), (0, 0, 1, 0, 1), (1, 0, 0, 3, 1), (1, 0, 1, 3, 1)}

In [46]:
tita, gama, landa

(Congruence([|0, 1|, |2, 3, 4, 5|, |6, 7, 8, 9|, |10, 11|, |12, 13, 14, 15|]),
 Congruence([|0, 10|, |1, 11|, |2, 3, 6, 7, 12, 13|, |4, 5, 8, 9, 14, 15|]),
 Congruence([|0, 1, 2, 4, 6, 8, 10, 11, 12, 14|, |3, 5, 7, 9, 13, 15|]))

In [47]:
from folpy.semantics.lattices import LatticeQuotient

LatticeQuotient(A_inicial, gama).draw()
LatticeQuotient(A_inicial, tita).draw()
LatticeQuotient(A_inicial, landa).draw()

### Caso 5

 - $F_1 := \{ 0, 1, 3 \}$
 - $F_2 := \{ 2 \}$
 - $F_2 := \{ 0, 4 \}$

In [48]:
Fs = {}
Fs[1] = {0, 1, 3}
Fs[2] = {2}
Fs[3] = {0, 4}
tita = reduce(lambda x, y: x & y, [deltas[i] for i in Fs[1]])
gama = reduce(lambda x, y: x & y, [deltas[i] for i in Fs[2]])
landa = reduce(lambda x, y: x & y, [deltas[i] for i in Fs[3]])

H_prima = [(i,j) for (i,j) in H if any(i in Fs[k] and j in Fs[k] for k in Fs.keys())]

A_barra_prima = P.universe
for (i, j) in H_prima:
    A_barra_prima = [x for x in A_barra_prima if x in dos_proyeccion_extendida(homeo, i, j)]

set(A_barra_prima).difference(set(A_barra))

{(0, 0, 0, 0, 1), (0, 0, 1, 0, 1), (1, 0, 0, 3, 1), (1, 0, 1, 3, 1)}

In [49]:
tita, gama, landa

(Congruence([|0, 1|, |2, 3, 4, 5|, |6, 7, 8, 9|, |10, 11|, |12, 13, 14, 15|]),
 Congruence([|0, 2, 3, 6, 7, 10, 12, 13|, |1, 4, 5, 8, 9, 11, 14, 15|]),
 Congruence([|0, 1, 2, 4, 6, 8|, |3, 5, 7, 9|, |10, 11, 12, 14|, |13, 15|]))

In [50]:
from folpy.semantics.lattices import LatticeQuotient

LatticeQuotient(A_inicial, gama).draw()
LatticeQuotient(A_inicial, tita).draw()
LatticeQuotient(A_inicial, landa).draw()

### Caso 6

 - $F_1 := \{ 0, 1, 3 \}$
 - $F_2 := \{ 2 \}$
 - $F_2 := \{ 1, 4 \}$

In [51]:
Fs = {}
Fs[1] = {0, 1, 3}
Fs[2] = {2}
Fs[3] = {1, 4}
tita = reduce(lambda x, y: x & y, [deltas[i] for i in Fs[1]])
gama = reduce(lambda x, y: x & y, [deltas[i] for i in Fs[2]])
landa = reduce(lambda x, y: x & y, [deltas[i] for i in Fs[3]])

H_prima = [(i,j) for (i,j) in H if any(i in Fs[k] and j in Fs[k] for k in Fs.keys())]

A_barra_prima = P.universe
for (i, j) in H_prima:
    A_barra_prima = [x for x in A_barra_prima if x in dos_proyeccion_extendida(homeo, i, j)]

set(A_barra_prima).difference(set(A_barra))

set()

In [52]:
tita, gama, landa

(Congruence([|0, 1|, |2, 3, 4, 5|, |6, 7, 8, 9|, |10, 11|, |12, 13, 14, 15|]),
 Congruence([|0, 2, 3, 6, 7, 10, 12, 13|, |1, 4, 5, 8, 9, 11, 14, 15|]),
 Congruence([|0, 1, 10, 11|, |2, 4, 6, 8, 12, 14|, |3, 5, 7, 9, 13, 15|]))

In [53]:
from folpy.semantics.lattices import LatticeQuotient

LatticeQuotient(A_inicial, gama).draw()
LatticeQuotient(A_inicial, tita).draw()
LatticeQuotient(A_inicial, landa).draw()

### Caso 7

 - $F_1 := \{ 0, 1, 3 \}$
 - $F_2 := \{ 1, 2, 4\}$

In [54]:
Fs = {}
Fs[1] = {0, 1, 3}
Fs[2] = {1, 2, 4}
tita = reduce(lambda x, y: x & y, [deltas[i] for i in Fs[1]])
gama = reduce(lambda x, y: x & y, [deltas[i] for i in Fs[2]])

H_prima = [(i,j) for (i,j) in H if any(i in Fs[k] and j in Fs[k] for k in Fs.keys())]

A_barra_prima = P.universe
for (i, j) in H_prima:
    A_barra_prima = [x for x in A_barra_prima if x in dos_proyeccion_extendida(homeo, i, j)]

set(A_barra_prima).difference(set(A_barra))

set()

In [55]:
tita, gama

(Congruence([|0, 1|, |2, 3, 4, 5|, |6, 7, 8, 9|, |10, 11|, |12, 13, 14, 15|]),
 Congruence([|0, 10|, |1, 11|, |2, 6, 12|, |3, 7, 13|, |4, 8, 14|, |5, 9, 15|]))

In [57]:
from folpy.semantics.lattices import LatticeQuotient

LatticeQuotient(A_inicial, gama).draw()
LatticeQuotient(A_inicial, tita).draw()